In [1]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import mysql.connector


2023-03-05 17:18:42.776373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 17:18:42.861844: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-05 17:18:42.861860: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-05 17:18:43.350622: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
# connect to database
mydb = mysql.connector.connect(
    user="root", passwd="jarvis@admin", host="192.168.1.97", port="3306")
connector = mydb.cursor()

In [3]:
deva_images_list = tf.data.Dataset.list_files(
    '/mnt/coding-drive/DATASETS/positive_face_dataset/Devasheesh/*.jpg', shuffle=True)
swar_images_list = tf.data.Dataset.list_files(
    '/mnt/coding-drive/DATASETS/positive_face_dataset/Swarnim/*.jpg', shuffle=True)
neg_images_list = tf.data.Dataset.list_files(
    '/mnt/coding-drive/DATASETS/negative-faces/VGG-Face2/data/test/*/*.jpg', shuffle=True)

# combine all the images into one dataset
images_list = deva_images_list.concatenate(swar_images_list).concatenate(neg_images_list)

# shuffle the dataset
images_list = images_list.shuffle(buffer_size=1000)

2023-03-05 17:18:46.711985: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-05 17:18:46.712004: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-05 17:18:46.712018: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (training-server): /proc/driver/nvidia/version does not exist
2023-03-05 17:18:46.712230: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# take the image path and return the label
def get_label(image_path):
    if 'Devasheesh' in image_path.numpy().decode('utf-8'):
        return 1
    elif 'Swarnim' in image_path.numpy().decode('utf-8'):
        return 2
    elif 'negative-faces' in image_path.numpy().decode('utf-8'):
        return 0
    else:
        import sys
        sys.exit('Error: Label not found')


In [10]:
#! see values inside variables
for i in images_list.take(5):
    print(i.numpy())
    print(get_label(i))

# print(len(images_list))
# print(type(images_list))

b'/mnt/coding-drive/DATASETS/negative-faces/VGG-Face2/data/test/n001439/0247_02.jpg'
0
b'/mnt/coding-drive/DATASETS/negative-faces/VGG-Face2/data/test/n008037/0558_02.jpg'
0
b'/mnt/coding-drive/DATASETS/negative-faces/VGG-Face2/data/test/n000129/0307_01.jpg'
0
b'/mnt/coding-drive/DATASETS/negative-faces/VGG-Face2/data/test/n004333/0156_01.jpg'
0
b'/mnt/coding-drive/DATASETS/positive_face_dataset/Devasheesh/Devasheesh_12.jpg'
1


In [ ]:
def extract_face_sql(path):
    # print(path)
    connector.execute("SELECT face FROM face_recognition.faces WHERE path = %s", (path,))
    face = connector.fetchone()
    if face is None:
        return None
    else:
        return face[0]

In [ ]:
# takes the image path and returns the the face as a tensor
def preprocess_image(image_path):
    label, path = get_label(image_path)
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [360, 480])
    # ? extract the face from the image using dataframe
    X, Y, W, H = extract_face_sql(path)
    
    face = tf.image.crop_to_bounding_box(image, X, Y, W, H)
    
    face = tf.cast(face, tf.float32)
    return face / 255.0, label
    # return image

In [ ]:
image_path, face, label = preprocess_image('data/Unknown/n000029/0145_01.jpg')
image_path

# image.numpy().shape
# print(type(preprocess_image('data\\Unknown\\n000029\\0145_01.jpg')))
# plt.imshow(image, cmap='gray')
# plt.title(label)
# image

In [ ]:
# dataset object

# Define a generator function that yields each file path
# def generator():
#     for filepath in imagesHAMK_list:
#         yield filepath

# dataset = tf.data.Dataset.from_generator(generator, output_types=tf.string)
# print(dataset)
dataset = images_list.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
for img, label in dataset.take(3):
    # print(img.shape)
    plt.imshow(img, cmap='gray')
    # print(type(label))
    plt.title(label)
    plt.show()